In [2]:
import numpy as np
from scipy import stats 
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker

In [3]:
engine = create_engine("postgresql:///music", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
con = engine.connect()

2019-05-08 08:53:36,505 INFO sqlalchemy.engine.base.Engine select version()
2019-05-08 08:53:36,507 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,509 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-08 08:53:36,510 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,512 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-08 08:53:36,512 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,514 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-08 08:53:36,515 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 08:53:36,516 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-05-08 08:53:36,516 INFO sqlalchemy.engine.base.Engine {}


In [11]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE pub_date NOT IN (SELECT pub_date FROM reviews WHERE pub_date < '2014-01-01' AND pub_date > '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]
len(result)

2019-05-08 08:57:14,900 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE pub_date NOT IN (SELECT pub_date FROM reviews WHERE pub_date < '2014-01-01' AND pub_date > '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 08:57:14,901 INFO sqlalchemy.engine.base.Engine {}


715

In [29]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]

2019-05-08 09:34:25,409 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date < '2010-01-01')
    AND artist in (SELECT artist FROM reviews where pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 09:34:25,409 INFO sqlalchemy.engine.base.Engine {}


[('john coltrane', 3, 9.5),
 ('gas', 3, 9.23333326975504),
 ('otis redding', 3, 9.0),
 ('the avalanches', 2, 9.0),
 ('this heat', 3, 9.0),
 ('sleater-kinney', 5, 8.86000003814697),
 ('albert ayler', 2, 8.84999990463257),
 ('caetano veloso', 5, 8.83999996185303),
 ('pavement', 4, 8.75),
 ('sly and the family stone', 2, 8.70000004768372),
 ('life without buildings', 2, 8.69999980926514),
 ('real estate', 2, 8.65000009536743),
 ('dr. dre', 2, 8.65000009536743),
 ('captain beefheart', 2, 8.65000009536743),
 ('loretta lynn', 2, 8.65000009536743),
 ('joanna newsom', 4, 8.64999985694885),
 ('pulp', 3, 8.63333320617676),
 ('fennesz', 7, 8.62857137407575),
 ('blur', 5, 8.62000007629394)]

In [31]:
query = """
    SELECT artist, count(reviews), avg(score)
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date < '2010-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    """
result = con.execute(query).fetchall()
result[1:20]

2019-05-08 09:35:23,803 INFO sqlalchemy.engine.base.Engine 
    SELECT artist, count(reviews), avg(score), avg(score WHERE pub_date < '2010-01-01')
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date < '2010-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    
2019-05-08 09:35:23,804 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 09:35:23,806 INFO sqlalchemy.engine.base.Engine ROLLBACK


ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "WHERE"
LINE 2: ...ECT artist, count(reviews), avg(score), avg(score WHERE pub_...
                                                             ^

[SQL: 
    SELECT artist, count(reviews), avg(score), avg(score WHERE pub_date < '2010-01-01')
    FROM reviews
    WHERE (pub_date < '2010-01-01'
    OR pub_date > '2014-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date < '2010-01-01')
    AND artist in 
        (SELECT artist 
        FROM reviews 
        WHERE pub_date > '2014-01-01')
    GROUP BY artist
    ORDER BY avg(score) DESC
    ]
(Background on this error at: http://sqlalche.me/e/f405)